In [4]:
from qutip import *
from pylab import *
from scipy import integrate
import time
from numpy import *
import multiprocessing as mp
#qutip.settings.num_cpus = 12

In [5]:
def mean_number_photons(delta,input_list,times,psi0):

    """
    This function handles the time evolution and returns the mean number 
    of photons for each value of detuning.
    """
    
    Nat = input_list[0]
    N = input_list[1] 
    kappa = input_list[2]
    g = input_list[3]
    EM = input_list[4]
    Oc = input_list[5]
    norm = input_list[6]
    
    G31 = input_list[7]
    G32 = input_list[8]
    D1 = input_list[9]
    D2 = input_list[10]
    
    #field operator
    op_list = []
    for m in range(Nat+1):
        op_list.append(qeye(3))
    op_list[0] = destroy(N)
    a = tensor(op_list) #muda conforme o número de átoomos
    
    #atomic operators
    S11_list = []
    S22_list = []
    S33_list = []
    S13_list = []
    S23_list = []
    
    for n in range(Nat):
        op_list = []
        for m in range(Nat+1):
            op_list.append(qeye(3))

        op_list[0] = qeye(N)
        
        op_list[n+1] = basis(3,0)*basis(3,0).dag()
        S11_list.append(tensor(op_list))

        op_list[n+1] = basis(3,1)*basis(3,1).dag()
        S22_list.append(tensor(op_list))

        op_list[n+1] = basis(3,2)*basis(3,2).dag()
        S33_list.append(tensor(op_list))
        
        op_list[n+1] = basis(3,0)*basis(3,2).dag()
        S13_list.append(tensor(op_list))
        
        op_list[n+1] = basis(3,1)*basis(3,2).dag()
        S23_list.append(tensor(op_list))        
       
    #total atomic operators
    S11 = 0
    S22 = 0
    S33 = 0
    S13 = 0
    S23 = 0
    
    for n in range(Nat):
        S11 = S11 + S11_list[n]
        S22 = S22 + S22_list[n]
        S33 = S33 + S33_list[n]
        S13 = S13 + S13_list[n]
        S23 = S23 + S23_list[n]

    #colapse Operators
    C = sqrt(kappa)*a   #cavity  
    C31_list = []
    C32_list = []
    
    for n in range(Nat):
        C31_list.append(sqrt(G31)*S13_list[n])
        C32_list.append(sqrt(G32)*S23_list[n])
    
    Clist = [C] + C31_list + C32_list
    
    #Hamiltonian    
    #Hp = (EM/2)*(a + a.dag())
    #Hi = g*(a*S13.dag() + a.dag()*S13)
    #Hc = Oc*(S23 + S23.dag())
    #H0 = D1*S33 + (D1-D2)*S22 + delta*S11 - delta*(a.dag()*a)

    # H_total     
    #Ht = H0 + Hp + Hi + Hc
    Ht = D1*S33 + (D1-D2)*S22 + delta*S11 - delta*a.dag()*a + (g*a*S13.dag() + Oc*S23.dag() + EM*a + g*a.dag()*S13 + Oc*S23 + EM*a.dag())
        

    
    # monte carlo solver
    #data = mcsolve(Ht, psi0, times, Clist, [a.dag() * a], progress_bar=False,ntraj = 24)
    #nm_at = data.expect[0]/norm
    #nm_a = nm_at[-1]
    
#steady state method (demands a high amount of memory as we increase the system's dimension)
    rho_a = steadystate(Ht, Clist, method='eigen', use_rcm=True, maxiter=1000, tol=1e-12)
    nm_a = expect(a.dag()*a, rho_a)/norm  # expectation value of nm -> normalized

    return nm_a

def width_full(NMa,delta_list,Oc):
    
    #maximum value of expected mean number of photons
    high_a = max(NMa) 
    
    #width calculation    
    for i in range(len(delta_list)):
        if NMa[i]>high_a/2:
            width_a = 2*(delta_list[i])
            
    #saving data
    Transmission_data = np.vstack((delta_list,NMa))
    file_data_store('Oc=%.2f.dat'%Oc, Transmission_data.T, numtype="real", numformat="decimal", sep= " ") 
        
    return width_a

def width_break(input_list,delta_list,times,psi0):
    
    for delta in delta_list:
            
            NMa = mean_number_photons(delta, input_list, times, psi0)
            
            if (delta == 0):
                high_a = NMa
    
            if (NMa > high_a/2):
                width_a = 2*(delta)
            else:
                wid = width_a
                break
    
    return wid

In [6]:
if __name__ == '__main__':   
    
    ini = time.time() #starts counting time
    print(time.ctime()) #print initial time

    nsteps1 = 600 #number of steps in the parameter list
    Oc_list = linspace(10**(-3), 5.0, nsteps1) #Rabi frequency list 
    index_j = range(nsteps1) #length of the parameter list

    nsteps = 1200 #number of steps for the detuning list
    delta_list = linspace(0, 2.5, nsteps) #detuning list
    index_i = range(nsteps) #length of the detuning list

    Nat = 1    
    N = 4 #Fock space dimension
    kappa = 1.0 #cavity mode dissipation rate (this is why the parameters should be passed as arguments to the function func1)
    g = (5.0/sqrt(Nat))*kappa #atom-field coupling strenght
    EM = sqrt(0.1)*kappa #pump field strength
    norm = (1.0*kappa)*(EM/kappa)**2 #normalization

    #decay rates
    G31 = 0.5*kappa  #atom
    G32 = 0.5*kappa  #atom

    #detunings
    D1 = 0.0
    D2 = 0.0
        
    times = np.linspace(0.0, 150.0, 250) #evolution time list

    aux_list = []
    for m in range(Nat+1):
        aux_list.append(basis(3,0))
    aux_list[0] = basis(N,0)
    psi0 = tensor(aux_list)
    
    #create empty lists to store the calculated FWHMs
    Wa = []
    
    for j in index_j:
        
        #transmission curve
        Oc = Oc_list[j]
        
        input_list = [Nat,N,kappa,g,EM,Oc,norm,G31,G32,D1,D2]
        
        #NMa = [mean_number_photons(delta, input_list, times, psi0) for delta in delta_list]
        #width_a = width_full(NMa,delta_list,Oc)
        #Wa.append(width_a)
        
        # ##or       
        width_a = width_break(input_list, delta_list, times, psi0)
        Wa.append(width_a)
        
            
        #progress tracking
        fin = time.time() 
        print("%f %f %.2f" % (Oc,width_a,(fin-ini)/3600))     
        
        
    output_data = np.vstack((Oc_list, Wa))   # join Oc and expt data
    file_data_store('test.dat', output_data.T, numtype="real", numformat="decimal", sep= " ") 
    

Tue Feb 22 12:42:06 2022
0.001000 1.017515 0.00
0.009346 1.017515 0.00
0.017691 1.013344 0.00
0.026037 1.005004 0.01
0.034382 0.992494 0.01
0.042728 0.979983 0.01
0.051073 0.963303 0.01
0.059419 0.946622 0.01
0.067765 0.921601 0.01
0.076110 0.900751 0.01
0.084456 0.875730 0.01
0.092801 0.846539 0.02
0.101147 0.821518 0.02
0.109492 0.792327 0.02
0.117838 0.763136 0.02
0.126184 0.733945 0.02
0.134529 0.708924 0.02
0.142875 0.683903 0.02
0.151220 0.658882 0.02
0.159566 0.638032 0.02
0.167912 0.617181 0.02
0.176257 0.600500 0.03
0.184603 0.583820 0.03
0.192948 0.571309 0.03
0.201294 0.558799 0.03
0.209639 0.546289 0.03
0.217985 0.533778 0.03
0.226331 0.525438 0.03
0.234676 0.517098 0.03
0.243022 0.508757 0.03
0.251367 0.500417 0.03
0.259713 0.492077 0.03
0.268058 0.487907 0.03
0.276404 0.483736 0.03
0.284750 0.475396 0.03
0.293095 0.471226 0.04
0.301441 0.467056 0.04
0.309786 0.462886 0.04
0.318132 0.458716 0.04
0.326477 0.458716 0.04
0.334823 0.454545 0.04
0.343169 0.450375 0.04
0.351514 

2.972025 0.483736 0.22
2.980371 0.483736 0.22
2.988716 0.483736 0.22
2.997062 0.483736 0.22
3.005407 0.483736 0.22
3.013753 0.483736 0.23
3.022098 0.487907 0.23
3.030444 0.487907 0.23
3.038790 0.487907 0.23
3.047135 0.487907 0.23
3.055481 0.487907 0.23
3.063826 0.487907 0.23
3.072172 0.487907 0.23
3.080518 0.487907 0.23
3.088863 0.492077 0.23
3.097209 0.492077 0.23
3.105554 0.492077 0.23
3.113900 0.492077 0.23
3.122245 0.492077 0.23
3.130591 0.492077 0.23
3.138937 0.492077 0.24
3.147282 0.492077 0.24
3.155628 0.496247 0.24
3.163973 0.496247 0.24
3.172319 0.496247 0.24
3.180664 0.496247 0.24
3.189010 0.496247 0.24
3.197356 0.496247 0.24
3.205701 0.496247 0.24
3.214047 0.496247 0.24
3.222392 0.500417 0.24
3.230738 0.500417 0.24
3.239083 0.500417 0.24
3.247429 0.500417 0.24
3.255775 0.500417 0.24
3.264120 0.500417 0.25
3.272466 0.500417 0.25
3.280811 0.500417 0.25
3.289157 0.504587 0.25
3.297503 0.504587 0.25
3.305848 0.504587 0.25
3.314194 0.504587 0.25
3.322539 0.504587 0.25
3.330885 0.